# 🕵️‍♂️ Práctica 2: Detective Sentimental – ¿Quién escribió este tuit?
Clasifica tweets según el autor usando características de estilo y sentimiento.

In [ ]:
# Prepara el entorno  # [Explica esta línea]
!pip install -q scikit-learn pandas matplotlib
from sklearn.feature_extraction.text import TfidfVectorizer  # Importa librerías necesarias
from sklearn.naive_bayes import MultinomialNB  # Importa librerías necesarias
from sklearn.model_selection import train_test_split  # Importa librerías necesarias
from sklearn.metrics import classification_report, confusion_matrix  # Importa librerías necesarias
import pandas as pd  # Importa librerías necesarias
import matplotlib.pyplot as plt  # Importa librerías necesarias

He descargado 4 datasets de Kaggle de 4 personas famosas porque son los que más datos tienen: <u>Bill Gates, Elon Musk, Taylor Sweet y Donald Trump</u>. Son datasets un poco antiguos, lo cual me parece mejor porque así puedo probar el código con los tweets nuevos que tengan actualmente.

Para poder trabajar con ellos los he subido a mi carpeta content de mi Colab. Al cerrar la sesión se pierden dichos datos pero en caso de que te hicieran falta para comprobar el código los he subido a mi carpeta de Drive:
[Datasets_Tweets](https://https://drive.google.com/drive/folders/1iGiK9Taf9XhBFqPw6FrZLPjHAiqPyjXV?usp=sharing)

```
https://drive.google.com/drive/folders/1iGiK9Taf9XhBFqPw6FrZLPjHAiqPyjXV?usp=sharing
```



In [ ]:
import pandas as pd

# --- Cargar y preparar datos de Bill Gates ---
nombre_archivo_gates = '/content/datos_tweets_BillGates.csv'  # Reemplaza con el nombre real
df_gates = pd.read_csv(nombre_archivo_gates, usecols=['text'])
df_gates['author'] = 'Bill Gates'
df_gates = df_gates.dropna(subset=['text'])
print("Primeras filas de Bill Gates:")
print(df_gates.head())
print("\n")

# --- Cargar y preparar datos de Elon Musk ---
nombre_archivo_elon = '/content/datos_tweets_elonmusk.csv'  # Reemplaza con el nombre real
df_elon = pd.read_csv(nombre_archivo_elon)
df_elon_seleccionado = df_elon[['text', 'screen_name']]
df_elon_seleccionado = df_elon_seleccionado.rename(columns={'screen_name': 'author'})
df_elon_seleccionado = df_elon_seleccionado.dropna(subset=['text'])
print("Primeras filas de Elon Musk:")
print(df_elon_seleccionado.head())
print("\n")

# --- Cargar y preparar datos de Donald Trump ---
nombre_archivo_trump = '/content/realdonaldtrump.csv'  # Reemplaza con el nombre real
df_trump = pd.read_csv(nombre_archivo_trump)
df_trump_seleccionado = df_trump[['content']]
df_trump_seleccionado = df_trump_seleccionado.rename(columns={'content': 'text'})
df_trump_seleccionado['author'] = 'Donald Trump'
df_trump_seleccionado = df_trump_seleccionado.dropna(subset=['text'])
print("Primeras filas de Donald Trump:")
print(df_trump_seleccionado.head())
print("\n")

# --- Cargar y preparar datos de Taylor Swift ---
nombre_archivo_swift = '/content/TaylorSwift13.csv'  # Reemplaza con el nombre real
df_swift = pd.read_csv(nombre_archivo_swift)
df_swift_seleccionado = df_swift[['content']]
df_swift_seleccionado = df_swift_seleccionado.rename(columns={'content': 'text'})
df_swift_seleccionado['author'] = 'Taylor Swift'
df_swift_seleccionado = df_swift_seleccionado.dropna(subset=['text'])
print("Primeras filas de Taylor Swift:")
print(df_swift_seleccionado.head())
print("\n")

# --- Combinar todos los DataFrames en uno solo ---
df_combinado = pd.concat([df_gates, df_elon_seleccionado, df_trump_seleccionado, df_swift_seleccionado], ignore_index=True)

# Mostrar información del DataFrame combinado
print("Información del DataFrame combinado:")
print(df_combinado.info())

# Mostrar las primeras filas del DataFrame combinado
print("\nPrimeras filas del DataFrame combinado:")
print(df_combinado.head())

Primeras filas de Bill Gates:
                                                text      author
0  Tanzania hopes to eliminate one of the world’s...  Bill Gates
1  By investing in R&amp;D, I know we can discove...  Bill Gates
2  This unusual tool for fighting disease turns o...  Bill Gates
3  My first trip to Africa with Melinda was the c...  Bill Gates
4  “The death that didn’t happen is not visible."...  Bill Gates


Primeras filas de Elon Musk:
                                                text    author
0  "If one day, my words are against science, cho...  elonmusk
1  I placed the flowers\r\n\r\nThree broken ribs\...  elonmusk
2          Atatürk Anıtkabir https://t.co/al3wt0njr6  elonmusk
3         @Bob_Richards One rocket, slightly toasted  elonmusk
4  @uncover007 500 ft so far. Should be 2 miles l...  elonmusk


Primeras filas de Donald Trump:
                                                text        author
0  Be sure to tune in and watch Donald Trump on L...  Donald Trump
1  

Entrenamiento del modelo con los datos del dataset

In [ ]:



# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_combinado["text"], df_combinado["author"], test_size=0.33, random_state=42)

# Inicializar y ajustar el vectorizador TF-IDF en los datos de entrenamiento
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

# Transformar los datos de prueba usando el vectorizador ajustado
X_test_vec = vectorizer.transform(X_test)

# Inicializar y entrenar el modelo Multinomial Naive Bayes
modelo = MultinomialNB()
modelo.fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones = modelo.predict(X_test_vec)

# Mostrar el informe de clasificación
print("Informe de Clasificación:")
print(classification_report(y_test, predicciones))

# Mostrar la matriz de confusión como un DataFrame de pandas
matriz_confusion_df = pd.DataFrame(confusion_matrix(y_test, predicciones), index=modelo.classes_, columns=modelo.classes_)
print("\nMatriz de Confusión:")
print(matriz_confusion_df)

Informe de Clasificación:
              precision    recall  f1-score   support

  Bill Gates       0.50      0.00      0.00       704
Donald Trump       0.89      1.00      0.94     14287
Taylor Swift       1.00      0.03      0.07       238
    elonmusk       1.00      0.03      0.07       900

    accuracy                           0.89     16129
   macro avg       0.85      0.27      0.27     16129
weighted avg       0.88      0.89      0.84     16129


Matriz de Confusión:
              Bill Gates  Donald Trump  Taylor Swift  elonmusk
Bill Gates             1           703             0         0
Donald Trump           0         14287             0         0
Taylor Swift           0           230             8         0
elonmusk               1           868             0        31


Vamos a ver cuántos valores hay en cada csv para saber por qué los resultados para Taylor Sweet son malos.

In [ ]:
print(df_combinado['author'].value_counts())

author
Donald Trump    43352
elonmusk         2678
Bill Gates       2087
Taylor Swift      756
Name: count, dtype: int64


Como Trump tiene un número mayor de datos vamos a hacer un <u>submuestreo</u> para mejorar los datos.

In [ ]:
# Obtener el número de tweets por autor
conteo_autores = df_combinado['author'].value_counts()
print("Número de tweets por autor antes del submuestreo:")
print(conteo_autores)

# Definir el tamaño objetivo para la clase mayoritaria (igual al siguiente más grande)
tamaño_objetivo = conteo_autores['elonmusk']

# Separar los tweets de la clase mayoritaria (Donald Trump)
df_trump = df_combinado[df_combinado['author'] == 'Donald Trump']

# Submuestrear aleatoriamente los tweets de Donald Trump
df_trump_submuestreado = df_trump.sample(n=tamaño_objetivo, random_state=42)

# Separar los tweets de las otras clases
df_no_trump = df_combinado[df_combinado['author'] != 'Donald Trump']

# Combinar el submuestreo de Donald Trump con los otros tweets
df_balanceado = pd.concat([df_trump_submuestreado, df_no_trump], ignore_index=True)

# Mezclar el dataset balanceado para evitar cualquier ordenamiento
df_balanceado = df_balanceado.sample(frac=1, random_state=42).reset_index(drop=True)

print("\nNúmero de tweets por autor después del submuestreo:")
print(df_balanceado['author'].value_counts())

# Ahora podemos volver a dividir los datos y entrenar el modelo con el dataset balanceado
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(
    df_balanceado["text"], df_balanceado["author"], test_size=0.33, random_state=42
)

vectorizer_balanced = TfidfVectorizer()
X_train_vec_balanced = vectorizer_balanced.fit_transform(X_train_balanced)
X_test_vec_balanced = vectorizer_balanced.transform(X_test_balanced)

modelo_balanceado = MultinomialNB()
modelo_balanceado.fit(X_train_vec_balanced, y_train_balanced)
predicciones_balanceadas = modelo_balanceado.predict(X_test_vec_balanced)

print("\nInforme de Clasificación con Dataset Submuestreado:")
print(classification_report(y_test_balanced, predicciones_balanceadas))

matriz_confusion_balanceada_df = pd.DataFrame(
    confusion_matrix(y_test_balanced, predicciones_balanceadas),
    index=modelo_balanceado.classes_,
    columns=modelo_balanceado.classes_
)
print("\nMatriz de Confusión con Dataset Submuestreado:")
print(matriz_confusion_balanceada_df)

Número de tweets por autor antes del submuestreo:
author
Donald Trump    43352
elonmusk         2678
Bill Gates       2087
Taylor Swift      756
Name: count, dtype: int64

Número de tweets por autor después del submuestreo:
author
Donald Trump    2678
elonmusk        2678
Bill Gates      2087
Taylor Swift     756
Name: count, dtype: int64

Informe de Clasificación con Dataset Submuestreado:
              precision    recall  f1-score   support

  Bill Gates       0.84      0.94      0.89       681
Donald Trump       0.76      0.98      0.86       873
Taylor Swift       1.00      0.05      0.10       256
    elonmusk       0.89      0.81      0.85       896

    accuracy                           0.83      2706
   macro avg       0.87      0.70      0.67      2706
weighted avg       0.85      0.83      0.79      2706


Matriz de Confusión con Dataset Submuestreado:
              Bill Gates  Donald Trump  Taylor Swift  elonmusk
Bill Gates           637            29             0        

Ahora el recall de Taylor es muy malo por ser la que tiene menos datos así que vamos a hacer un <u>sobremuestreo</u> para sus datos:

In [ ]:
# Separar los tweets de Taylor Swift
df_swift_balanceado = df_balanceado[df_balanceado['author'] == 'Taylor Swift']

# Separar los tweets de los otros autores
df_no_swift_balanceado = df_balanceado[df_balanceado['author'] != 'Taylor Swift']

# Calcular cuántos tweets necesitamos duplicar de Taylor Swift
tamaño_objetivo_swift = df_balanceado['author'].value_counts().max()  # Usamos el tamaño de la clase mayoritaria actual
n_duplicar = tamaño_objetivo_swift - len(df_swift_balanceado)

# Duplicar aleatoriamente los tweets de Taylor Swift
df_swift_oversampled = df_swift_balanceado.sample(n=n_duplicar, replace=True, random_state=42)

# Combinar los tweets sobremuestreados de Taylor Swift con los otros tweets
df_oversampled = pd.concat([df_no_swift_balanceado, df_swift_oversampled, df_swift_balanceado], ignore_index=True)

# Mezclar el dataset sobremuestreado
df_oversampled = df_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

print("Número de tweets por autor después del sobremuestreo de Taylor Swift:")
print(df_oversampled['author'].value_counts())

# Dividir los datos sobremuestreados
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(
    df_oversampled["text"], df_oversampled["author"], test_size=0.33, random_state=42
)

# Vectorizar el texto
vectorizer_over = TfidfVectorizer()
X_train_vec_over = vectorizer_over.fit_transform(X_train_over)
X_test_vec_over = vectorizer_over.transform(X_test_over)

# Entrenar el modelo
modelo_over = MultinomialNB()
modelo_over.fit(X_train_vec_over, y_train_over)

# Hacer predicciones
predicciones_over = modelo_over.predict(X_test_vec_over)

# Evaluar el modelo
print("\nInforme de Clasificación con Sobremuestreo de Taylor Swift:")
print(classification_report(y_test_over, predicciones_over))

matriz_confusion_over_df = pd.DataFrame(
    confusion_matrix(y_test_over, predicciones_over),
    index=modelo_over.classes_,
    columns=modelo_over.classes_
)
print("\nMatriz de Confusión con Sobremuestreo de Taylor Swift:")
print(matriz_confusion_over_df)

Número de tweets por autor después del sobremuestreo de Taylor Swift:
author
Donald Trump    2678
elonmusk        2678
Taylor Swift    2678
Bill Gates      2087
Name: count, dtype: int64

Informe de Clasificación con Sobremuestreo de Taylor Swift:
              precision    recall  f1-score   support

  Bill Gates       0.96      0.91      0.93       678
Donald Trump       0.91      0.96      0.93       912
Taylor Swift       0.87      0.99      0.93       905
    elonmusk       0.97      0.81      0.88       845

    accuracy                           0.92      3340
   macro avg       0.93      0.92      0.92      3340
weighted avg       0.92      0.92      0.92      3340


Matriz de Confusión con Sobremuestreo de Taylor Swift:
              Bill Gates  Donald Trump  Taylor Swift  elonmusk
Bill Gates           614             9            48         7
Donald Trump           1           875            23        13
Taylor Swift           0             5           899         1
elonmusk 

Ahora que estoy contento con los resultados que me da el modelo voy a hacer pruebas con él. Lo primero ha sido copiar varios tweets actualizados de cado uno de los autores para que luego me diga de quién es cada uno.

In [ ]:
t_bill_1 = """
Halting funding for the World Health Organization during a world health crisis is as dangerous as it sounds. Their work is slowing the spread of COVID-19 and if that work is stopped no other organization can replace them. The world needs @WHO now more than ever.Some memories stick with you forever. Others follow you around the internet for 28 years. Happy birthday, @Windows.
"""
t_bill_2 = """
Congratulations to @narendramodi on winning a third term as Prime Minister. You have strengthened India's position as a source of innovation for global progress in sectors like health, agriculture, women-led development, and digital transformation. Look forward to a continued partnership to enhance the lives of people across India and the world. @PMOIndia
"""
t_bill_3 = """
My upcoming memoir Source Code is all about the lessons and experiences that laid the foundation for everything in my life that followed. https://gatesnot.es/source-code-tw
"""


t_taylor_1 = """
The Tortured Poets Department. An anthology of new works that reflect events, opinions and sentiments from a fleeting and fatalistic moment in time - one that was both sensational and sorrowful in equal measure. This period of the author’s life is now over, the chapter closed and boarded up. There is nothing to avenge, no scores to settle once wounds have healed. And upon further reflection, a good number of them turned out to be self-inflicted. This writer is of the firm belief that our tears become holy in the form of ink on a page. Once we have spoken our saddest story, we can be free of it.
"""
t_taylor_2 = """
And then all that’s left behind is the tortured poetry.
"""
t_taylor_3 = """
All’s fair in love and poetry... New album THE TORTURED POETS DEPARTMENT. Out April 19
"""
t_taylor_4 = """
It’s here. It’s yours, it’s mine, it’s ours. It’s an album I wrote alone about the whims, fantasies, heartaches,dramas and tragedies I lived out as a young woman between 18 and 20. I remember making tracklist after tracklist,obsessing over the right way to tell the story. I had to be ruthless with my choices, and I left behind some songs. I am still unfailingly proud of now. Therefore, you have 6 From The Vault tracks! I recorded this album when I was 32 (and still growing up, now) and the memories it brought back filled me with nostalgia and appreciation.For life, for you, for the fact that I get to reclaim my work. Thank you a million times, for the memories that break our fall. 💥🐉🏰  Speak Now (MY VERSION!) is out now.
"""



t_trump_1 = """
The United States has a chance to do something that should have been done DECADES AGO. Don’t be Weak! Don’t be Stupid! Don’t be a PANICAN (A new party based on Weak and Stupid people!). Be Strong, Courageous, and Patient, and GREATNESS will be the result!
"""
t_trump_2 = """
Oil prices are down, interest rates are down (the slow moving Fed should cut rates!), food prices are down, there is NO INFLATION, and the long time abused USA is bringing in Billions of Dollars a week from the abusing countries on Tariffs that are already in place. This is despite the fact that the biggest abuser of them all, China, whose markets are crashing, just raised its Tariffs by 34%, on top of its long term ridiculously high Tariffs (Plus!), not acknowledging my warning for abusing countries not to retaliate. They’ve made enough, for decades, taking advantage of the Good OL’ USA! Our past “leaders” are to blame for allowing this, and so much else, to happen to our Country. MAKE AMERICA GREAT AGAIN!
"""
t_trump_3 = """
These Houthis gathered for instructions on an attack. Oops, there will be no attack by these Houthis!
"""
t_trump_4 = """
They will never sink our ships again!
"""



t_elon_1 = """
The first astronauts to orbit the poles
"""
t_elon_2 = """
The long con of the left is corruption of the judiciary
"""
t_elon_3 = """
Easiest money you ever made!
"""
t_elon_4 = """
You don’t even need to be from Wisconsin, you just need to get people in the state to hold a picture of @TeamSchimel (can be on a phone/device or paper) in one hand and go thumbs up with the other hand.
"""
t_elon_5 = """
Every time you do that you get $20 and they get $20. The goal is to build awareness about the election tomorrow, so this offer applies to today and tomorrow.
"""
t_elon_6 = """
Vote for Superjudge Brad Schimel in Wisconsin on Tuesday!!
"""
t_elon_7 = """
The Republican House majority is razor thin and the Democrats want to redraw Wisconsin districts to flip the House and stop the government reforms.
"""


In [ ]:
# Le damos formato a los tweets
tweets_actuales = [
    t_bill_1.replace('\n', ' ').strip(),
    t_bill_2.replace('\n', ' ').strip(),
    t_bill_3.replace('\n', ' ').strip(),
    t_taylor_1.replace('\n', ' ').strip(),
    t_taylor_2.replace('\n', ' ').strip(),
    t_taylor_3.replace('\n', ' ').strip(),
    t_taylor_4.replace('\n', ' ').strip(),
    t_trump_1.replace('\n', ' ').strip(),
    t_trump_2.replace('\n', ' ').strip(),
    t_trump_3.replace('\n', ' ').strip(),
    t_trump_4.replace('\n', ' ').strip(),
    t_elon_1.replace('\n', ' ').strip(),
    t_elon_2.replace('\n', ' ').strip(),
    t_elon_3.replace('\n', ' ').strip(),
    t_elon_4.replace('\n', ' ').strip(),
    t_elon_5.replace('\n', ' ').strip(),
    t_elon_6.replace('\n', ' ').strip(),
    t_elon_7.replace('\n', ' ').strip()
]
bill = "Bill Gates"
taylor = "Taylor Swift"
trump = "Donald Trump"
elon = "Elon Musk"

# Y se crea un array con para sabes quién es el autor real de cada tweet coincidiendo la posición del tweet con el autor
autores_reales = [
    bill, bill, bill, bill,
    taylor, taylor, taylor, taylor,
    trump, trump, trump, trump,
    elon, elon, elon, elon, elon, elon, elon
]


In [ ]:
# Vectorizar los tweets actuales USANDO el vectorizador ENTRENADO
tweets_actuales_vec = vectorizer_over.transform(tweets_actuales)

# Realizar las predicciones
predicciones_actuales = modelo_over.predict(tweets_actuales_vec)

# Imprimir las predicciones junto con los tweets originales y los autores reales
print("Predicciones en Tweets Actuales:")
for i, tweet in enumerate(tweets_actuales):
    print(f"Tweet: \"{tweet[:60]}...\"") # Mostrar solo los primeros 60 caracteres
    print(f"Autor Real: {autores_reales[i]}")
    print(f"Predicción del Modelo: {predicciones_actuales[i]}")
    print("-" * 30)

Predicciones en Tweets Actuales:
Tweet: "Halting funding for the World Health Organization during a w..."
Autor Real: Bill Gates
Predicción del Modelo: Bill Gates
------------------------------
Tweet: "Congratulations to @narendramodi on winning a third term as ..."
Autor Real: Bill Gates
Predicción del Modelo: Bill Gates
------------------------------
Tweet: "My upcoming memoir Source Code is all about the lessons and ..."
Autor Real: Bill Gates
Predicción del Modelo: Bill Gates
------------------------------
Tweet: "The Tortured Poets Department. An anthology of new works tha..."
Autor Real: Bill Gates
Predicción del Modelo: Taylor Swift
------------------------------
Tweet: "And then all that’s left behind is the tortured poetry...."
Autor Real: Taylor Swift
Predicción del Modelo: Taylor Swift
------------------------------
Tweet: "All’s fair in love and poetry... New album THE TORTURED POET..."
Autor Real: Taylor Swift
Predicción del Modelo: Taylor Swift
---------------------------

### Conclusiones
Los resultados para Bill Gates y Taylor Swift han sido buenos. Pero con Elon Musk y Donald Trump ha habido varias confusiones entre ellos y eso se debe a que usan una dialectica muy parecida, de hecho como vemos en las noticias son dos personas muy afines

# ✍️ Preguntas de reflexión
- ¿Qué términos fueron más distintivos para cada autor?
- ¿Qué estrategias mejorarían la precisión del modelo?

Escribe tus respuestas aquí abajo:

Para saber los terminos más distintivos de cada autor usamos el siguiente código:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# --- Re-entrenar el modelo con el vectorizador ajustado (bigramas y stop words en inglés de scikit-learn) ---
X = df_oversampled["text"]
y = df_oversampled["author"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

vectorizer_ajustado_v3 = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
X_train_vec_ajustado_v3 = vectorizer_ajustado_v3.fit_transform(X_train)
X_test_vec_ajustado_v3 = vectorizer_ajustado_v3.transform(X_test)

modelo_ajustado_v3 = MultinomialNB()
modelo_ajustado_v3.fit(X_train_vec_ajustado_v3, y_train)

predicciones_test_ajustado_v3 = modelo_ajustado_v3.predict(X_test_vec_ajustado_v3)

print("Informe de Clasificación con Vectorizador Ajustado (Bigramas y Stop Words en Inglés):")
print(classification_report(y_test, predicciones_test_ajustado_v3))

matriz_confusion_ajustada_v3_df = pd.DataFrame(
    confusion_matrix(y_test, predicciones_test_ajustado_v3),
    index=modelo_ajustado_v3.classes_,
    columns=modelo_ajustado_v3.classes_
)
print("\nMatriz de Confusión con Vectorizador Ajustado (Stop Words en Inglés):")
print(matriz_confusion_ajustada_v3_df)

# --- Mostrar los términos más distintivos con el nuevo vectorizador ---
clases_v3 = modelo_ajustado_v3.classes_
top_words_df_v3 = top_n_words(modelo_ajustado_v3, vectorizer_ajustado_v3, clases_v3, n=20)

print("\nTérminos más distintivos por autor (con Stop Words en Inglés):")
for autor in clases_v3:
    top_words = top_words_df_v3[top_words_df_v3['class'] == autor].sort_values(by='weight', ascending=False)
    print(f"\n--- {autor} ---")
    for index, row in top_words.iterrows():
        print(f"{row['feature']}: {row['weight']:.4f}")

Informe de Clasificación con Vectorizador Ajustado (Bigramas y Stop Words en Inglés):
              precision    recall  f1-score   support

  Bill Gates       0.97      0.90      0.93       678
Donald Trump       0.92      0.96      0.94       912
Taylor Swift       0.89      1.00      0.94       905
    elonmusk       0.96      0.84      0.90       845

    accuracy                           0.93      3340
   macro avg       0.93      0.92      0.93      3340
weighted avg       0.93      0.93      0.93      3340


Matriz de Confusión con Vectorizador Ajustado (Stop Words en Inglés):
              Bill Gates  Donald Trump  Taylor Swift  elonmusk
Bill Gates           607            15            40        16
Donald Trump           0           880            16        16
Taylor Swift           0             2           903         0
elonmusk              21            62            51       711

Términos más distintivos por autor (con Stop Words en Inglés):

--- Bill Gates ---
http: -6.

### ¿Qué términos fueron más distintivos para cada autor?

Como vemos por los resultados, cada autor tiene unas palabras características:

<b>Bill Gates</b>:

world,people,polio,lives,book,help,progress,energy,read,africa, poverty, global,
fight y work.

Son palabras que tienen que ver con su labor filántropica

<b>Taylor Swift</b>:

thank, love, reputation, video, guys, like, tonight, time, new, album, song, version y life

Aquí podemos ver que habla mucho para sus admiradores y de su trabajo como cantante.

<b>Donald Trump</b>:

realdonaldtrump, great, trump, president, thanks, obama, america, country

En el caso de Donald se ve que son palabras para hablar de sí mismo y de política

<b>Elon Musk</b>

tesla, yes, model, good, rocket, spacex, yeah, thanks, falcon, launch, car, teslamotors, exactly, right, cars.

Se puede ver que Elon Musk se centra más en sus empresas.



### ¿Qué estrategias mejorarían la precisión del modelo?

En este caso como los datos que obtenido eran desiguales de tamaño, lo que he hecho con la ayuda inestimable de la IA ha sido hacer un submuestreo del dataset con mayor contidad de datos y sobremuestreo en el que menos, lo que al ver la matriz de confusión que genera el modelo después de entrenarlo ha mejorado mucho la precisión y la sensibilidad.

Aunque he tenido que rebuscar los datasets de personas conocidas, hubiera sido mejor haber tenido más datos y quizás usar personalidades algo más diferentes que la de Donald Trump y Elon Musk para así poder tener mejor porcentaje de acierto.

### Bibliografía

Como comentos varias veces en los textos, aunque he usado el modelo del ejercicio de ejemplo de detective sentimental, para poder hacer el ejercicio completo he tenido que usar la IA de Gemini para ir haciendo paso a paso lo que el ejercicio requería así como correcciones ya que los primeros resultados no eran óptimos con la autora Taylor Swift. También gracias a eso código he aprendido un poco más de cómo se puede identificar el texto de un autor.